In [1]:
%cd ..

/home/wiss/hofherrf/gitRepos/CodeRelease


In [2]:
import os
import yaml
import torch
from torchvision import utils
from models.sceneRepresentation import Scene
from dataset.dataset import ImageDataset_realData
from util.util import compute_psnr
import matplotlib.pyplot as plt

In [3]:
path_experiment = os.path.join(
    os.path.abspath(''),
    'experiments',
    '2023-01-30',
    'real_world',
    'pendulum',
    '10-13-05'
)

In [4]:
path_conf = os.path.join(path_experiment, '.hydra','config.yaml')
with open(path_conf) as f:
    cfg = yaml.safe_load(f)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

model = Scene(**cfg['scene']['background'])

model.add_pendulum(**cfg['ode'], **cfg['scene']['local_representation'])

path_ckpt = os.path.join(path_experiment, 'ckpt.pth')
model.load_state_dict(torch.load(path_ckpt))

model.use_homography = cfg['homography']['enable']

model.to(device)
print("Moved to device")

Moved to device


In [5]:
data_path = os.path.join(os.path.abspath(''), 'data',cfg['data']['path_data'])

train_data = ImageDataset_realData(
    **cfg['data'],
    normalize_by_H=True,
    max_samples=cfg['data']['samples_train']
)

eval_data = ImageDataset_realData(
    **cfg['data'],
    normalize_by_H=True,
    indices=train_data.unused_inds
)

tspan_eval = eval_data.t_steps.to(device)
print(f"Tspan eval data: {tspan_eval}")
print(f"Tspan train data: {train_data.t_steps}")

Tspan eval data: tensor([0.0000, 0.1000, 0.3000, 0.5000, 0.7000, 0.9000, 1.1000, 1.3000, 1.5000,
        1.7000, 1.9000, 2.0000, 2.1000, 2.2000, 2.3000, 2.4000, 2.5000, 2.6000,
        2.7000, 2.8000, 2.9000, 3.0000, 3.1000, 3.2000, 3.3000, 3.4000, 3.5000,
        3.6000, 3.7000, 3.8000, 3.9000], device='cuda:0')
Tspan train data: tensor([0.0000, 0.2000, 0.4000, 0.6000, 0.8000, 1.0000, 1.2000, 1.4000, 1.6000,
        1.8000])


In [6]:
# Render images
model.eval()
model.update_trafo(tspan_eval)
H, W = eval_data.get_image_dim()
output = model.render_image(W, H, normalize_by_H=True)
ims = output["Image"].cpu()
masks = output['Mask'].cpu()

psnr = compute_psnr(ims, eval_data.get_full_images())
norm = torch.norm(torch.eye(3) - model.homography_matrix.cpu())

measured_l = 0.271
rel_error_l = torch.abs(model.local_representation.ode.l_pendulum - measured_l) / measured_l
print(f"PSNR: {psnr}, Norm diff: {norm}")
print(f"Rel Error l: {100*rel_error_l:.2f}%")
print(f"{psnr:.2f} & {norm:.2f}")

PSNR: 29.40026092529297, Norm diff: 0.06202326714992523
Rel Error l: 4.09%
29.40 & 0.06


In [7]:
path_folder_rendering = os.path.join(path_experiment, 'renderingsRealWorld')
os.makedirs(path_folder_rendering)

width = int(ims.shape[2] / 2 - 2)

inds_to_save = [i for i in range(8, 15) if (i+1)%1==0]
print(f"Saving imgaes for timesteps {tspan_eval[inds_to_save]}")

# Save individual images
for i in inds_to_save:
    path = os.path.join(path_folder_rendering, f"{i}_eval.jpg")
    cur_im = ims[i].permute(2, 0, 1)
    utils.save_image(cur_im, path)

    path = os.path.join(path_folder_rendering, f"{i}_gt.jpg")
    cur_im_gt = eval_data.get_full_images(i).permute(2, 0, 1)
    utils.save_image(cur_im_gt, path)

    path = os.path.join(path_folder_rendering, f"{i}_mask_eval.jpg")
    cur_mask = masks[i]
    utils.save_image(cur_mask, path)

    path = os.path.join(path_folder_rendering, f"{i}_merged_gtRight.jpg")
    merged_im = torch.zeros_like(cur_im)
    merged_im[0, :, :] = 1.0
    merged_im[:, :, :width] = cur_im[:, :, :width]
    merged_im[:, :, -width:] = cur_im_gt[:, :, -width:]
    utils.save_image(merged_im, path)

Saving imgaes for timesteps tensor([1.5000, 1.7000, 1.9000, 2.0000, 2.1000, 2.2000, 2.3000],
       device='cuda:0')
